# Data Wrangling

In [1]:
import os
import sys
import glob
import datetime
import sqlite3
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series, DataFrame
from IPython.display import Image, HTML
from scipy import stats

%matplotlib inline
pd.options.display.mpl_style = 'default'
plt.style.use('ggplot')

print pd.__version__

%load_ext rpy2.ipython

0.15.1


# Addition Files contains Year (515576 songs for full 300GB, 4680 for 1.8GB subset)

In [2]:
getting_data_url = 'http://labrosa.ee.columbia.edu/millionsong/pages/getting-dataset'
HTML("<iframe src='%s' width=1000 height=333></iframe>"%getting_data_url)

# Text files in Additional Files directory

In [2]:
txt_files = !ls /media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/*txt
txt_files

['/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/mxm_reverse_mapping.txt',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_artist_location.txt',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_tracks_per_year.txt',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_unique_artists.txt',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_unique_mbtags.txt',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_unique_terms.txt',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_unique_tracks.txt']

In [6]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

for f in txt_files:
    print f
    preview = !head $f
    pp.pprint(preview)
    print '\n\n'

/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/mxm_reverse_mapping.txt
[   'd\xc3\xada<SEP>d\xc3\xada',
    'pido<SEP>pido',
    'hatr<SEP>hatred',
    'pide<SEP>pide',
    'yellow<SEP>yellow',
    'four<SEP>four',
    'sleev<SEP>sleeve',
    'sleep<SEP>sleep',
    'thirst<SEP>thirst',
    'upsid<SEP>upside']



/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_artist_location.txt
[   'AR00A6H1187FB5402A<SEP>42.73383<SEP>-84.59334<SEP>TRAKWGL12903CB8529<SEP>The Meatmen',
    'AR00MBZ1187B9B5DB1<SEP>29.95244<SEP>-90.05202<SEP>TRAFCPP128F426EC01<SEP>Memphis Minnie',
    'AR01IP11187B9AF5D2<SEP>28.33268<SEP>-80.73486<SEP>TRAZNKG12903CDCF8A<SEP>Call To Preserve',
    'AR01VU31187B997DA0<SEP>43.0026<SEP>-83.7826<SEP>TRAKZMB128F427B44F<SEP>Grand Funk',
    'AR01W2D1187FB5912F<SEP>40.01574<SEP>-105.27924<SEP>TRASHYD128F93119EE<SEP>3OH!3',
    'AR022JO1187B99587B<SEP>0.10974<SEP>113.9174<SEP>TRAHSJR12903CBF093<SEP>Ross',
    'AR02YGA1187B9B8AC4<SEP>-34.60852<SEP>-58.373

# Database DB SQLite3 files in Additional Files directory

In [8]:
db_files = !ls /media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/*db
db_files

['/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/mxm_dataset.db',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_artist_similarity.db',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_artist_term.db',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_metadata.db',
 '/media/1ADF-0E69/msong/MillionSongSubset/AdditionalFiles/subset_track_metadata.db']

# Lyrics Dataset musiXmatch

In [10]:
getting_lyrics_url = 'http://labrosa.ee.columbia.edu/millionsong/blog/11-4-11-musixmatch-dataset-connecting-lyrics'
HTML("<iframe src='%s' width=1000 height=333></iframe>"%getting_lyrics_url)

# Local Path Setup

In [2]:
msd_subset_path = os.path.join('/media/1ADF-0E69/msong', 'MillionSongSubset')
msd_subset_data_path = os.path.join(msd_subset_path, 'data')
msd_subset_addf_path = os.path.join(msd_subset_path, 'AdditionalFiles')
assert os.path.isdir(msd_subset_path), 'wrong_path'
msd_code_path = os.path.join('/media/1ADF-0E69/msong', 'MSongsDB')
assert os.path.isdir(msd_code_path)

sys.path.append(os.path.join(msd_code_path, 'PythonSrc'))
import hdf5_getters as GETTERS

subset_tracks_per_year_path = os.path.join(msd_subset_addf_path, 'subset_tracks_per_year.txt')
tracks_per_year_path = os.path.join('/media/1ADF-0E69/msong', 'tracks_per_year.txt')

#Create Custom Year Extended Dataset

In [28]:
!head $tracks_per_year_path ; wc -l $tracks_per_year_path

1922<SEP>TRSGHLU128F421DF83<SEP>Alberta Hunter<SEP>Don't Pan Me
1922<SEP>TRMYDFV128F42511FC<SEP>Barrington Levy<SEP>Warm And Sunny Day
1922<SEP>TRRAHXQ128F42511FF<SEP>Barrington Levy<SEP>Looking My Love
1922<SEP>TRFAFTK12903CC77B8<SEP>Barrington Levy<SEP>Warm And Sunny Day
1922<SEP>TRSTBUY128F4251203<SEP>Barrington Levy<SEP>Mandela You're Free
1922<SEP>TRODGCA128F4251206<SEP>Barrington Levy<SEP>Something In My Heart (Full Vocal)
1924<SEP>TRLOZBM128F4280BF6<SEP>Papa Charlie Jackson<SEP>Airy Man Blues
1924<SEP>TRZCBEO128F4285118<SEP>Vernon Dalhart<SEP>Wreck Of The Old 97
1924<SEP>TRTRZWN128F4285110<SEP>Vernon Dalhart<SEP>The Prisoner's Song
1924<SEP>TRTRKSF12903CFEDD7<SEP>Vernon Dalhart<SEP>Wreck Of The Old 97
515576 /media/1ADF-0E69/msong/tracks_per_year.txt


In [3]:
def trackid_to_filepath(track_id):
    topdir = track_id[2]
    subdir = track_id[3]
    subsubdir = track_id[4]
    
    track_filepath = os.path.join(msd_subset_data_path,
                                  os.path.join(topdir,
                                               os.path.join(subdir,
                                                            os.path.join(subsubdir, track_id + '.h5'))))
    return track_filepath

def get_timbre_avg(timbre_12_segment_matrix):
    return list(np.mean(timbre_12_segment_matrix, axis=0))

def get_pitch_avg(pitch_12_segment_matrix):
    return list(np.mean(pitch_12_segment_matrix, axis=0))

def get_record_line_string_by_grabbing_fields_from_h5(h5, year='', track_id='', artist_name='', title=''):
    yr = GETTERS.get_year(h5) if not year else year
    tid = GETTERS.get_track_id(h5) if not track_id else track_id
    aname = GETTERS.get_artist_name(h5) if not artist_name else artist_name
    ttitle = GETTERS.get_title(h5) if not title else title
    timbre_avg = get_timbre_avg(GETTERS.get_segments_timbre(h5))
    lat = GETTERS.get_artist_latitude(h5)
    lon = GETTERS.get_artist_longitude(h5)
    fam = GETTERS.get_artist_familiarity(h5)
    hot = GETTERS.get_artist_hotttnesss(h5)
    mbtags = GETTERS.get_artist_mbtags(h5)
    terms = GETTERS.get_artist_terms(h5)
    dance = GETTERS.get_danceability(h5)
    duration = GETTERS.get_duration(h5)
    energy = GETTERS.get_energy(h5)
    loudness = GETTERS.get_loudness(h5)
    mode = GETTERS.get_mode(h5)
    song_hotness = GETTERS.get_song_hotttnesss(h5)
    tempo = GETTERS.get_tempo(h5)
    release = GETTERS.get_release(h5)
    pitch_avg = get_pitch_avg(GETTERS.get_segments_pitches(h5))
    record = [yr, tid, aname, ttitle] + timbre_avg + [lat,lon, fam,
                                                      hot, mbtags, terms,
                                                      dance, duration, energy,
                                                      loudness, mode, song_hotness,
                                                      tempo, release] + pitch_avg
    record_as_delimited_string = '<SEP>'.join(map(str,record))
    h5.close()
    
    return record_as_delimited_string.replace('\n', ' ')

def create_extended_year_dataset_and_write_to_file(fname='tracks_per_year_extended.txt', overwrite=True):
    if os.path.isfile(fname) and overwrite:
        !rm $fname
        !rm tracks_out_of_subset.txt
        !touch $fname
        !touch tracks_out_of_subset.txt
        
    outfile = open(fname,'w')
        
    fields = ['year',
              'track_id',
              'artist_name',
              'title',
              'timbre_avg1_loudness',
              'timbre_avg2_brightness',
              'timbre_avg3_flatness',
              'timbre_avg4_attack',
              'timbre_avg5',
              'timbre_avg6',
              'timbre_avg7',
              'timbre_avg8',
              'timbre_avg9',
              'timbre_avg10',
              'timbre_avg11',
              'timbre_avg12',
              'artist_latitude',
              'artist_longitude',
              'artist_familiarity',
              'artist_hotttnesss',
              'artist_mbtags',
              'artist_terms',
              'danceability',
              'duration',
              'energy',
              'loudness',
              'mode',
              'song_hotttnesss',
              'tempo',
              'release',
              'pitch_avg1',
              'pitch_avg2',
              'pitch_avg3',
              'pitch_avg4',
              'pitch_avg5',
              'pitch_avg6',
              'pitch_avg7',
              'pitch_avg8',
              'pitch_avg9',
              'pitch_avg10',
              'pitch_avg11',
              'pitch_avg12']
    
    fields_as_sep_delimited_string = '<SEP>'.join(fields)
    outfile.write(fields_as_sep_delimited_string + '\n')
               
    with open(tracks_per_year_path, 'r') as f:
        for line in f:
            year, track_id, artist_name, title = line.split('<SEP>')
            path = trackid_to_filepath(track_id)
            
            if os.path.isfile(path):
                h5 = GETTERS.open_h5_file_read(path)
                outfile.write(get_record_line_string_by_grabbing_fields_from_h5(h5) + '\n')
            else:
                !echo $track_id >> tracks_out_of_subset.txt
                
        outfile.close()

Rewriting Header in CSV files

In [10]:
def replace_first_line(src_filename, target_filename, replacement_line):
    f = open(src_filename)
    first_line, remainder = f.readline(), f.read()
    t = open(target_filename,"w")
    t.write(replacement_line + "\n")
    t.write(remainder)
    t.close()

# Field List

In [64]:
field_list_url_src = 'http://labrosa.ee.columbia.edu/millionsong/pages/field-list'
HTML("<iframe src='%s' width=1000 height=333></iframe>"%field_list_url_src)

# Run Dataset Creation Code

In [9]:
%time create_extended_year_dataset_and_write_to_file()

rm: cannot remove `tracks_out_of_subset.txt': No such file or directory
CPU times: user 22min 40s, sys: 1h 30min 32s, total: 1h 53min 13s
Wall time: 18h 8min 19s


# Link Tracks per Year and UCI Datasets

Blind Lemon Jefferson, Got The Blues

In [65]:
path = trackid_to_filepath('TRBFMCN128F423625D')
h5 = GETTERS.open_h5_file_read(path)
timbre = GETTERS.get_segments_timbre(h5)
h5.close()

In [66]:
Cov = np.cov(timbre.T)
print Cov.shape

timbre_fields=['timbre_avg1_loudness',
              'timbre_avg2_brightness',
              'timbre_avg3_flatness',
              'timbre_avg4_attack',
              'timbre_avg5',
              'timbre_avg6',
              'timbre_avg7',
              'timbre_avg8',
              'timbre_avg9',
              'timbre_avg10',
              'timbre_avg11',
              'timbre_avg12']
DataFrame(Cov, index=timbre_fieldslds, columns=timbre_fields)

(12, 12)


,timbre_avg1_loudness,timbre_avg2_brightness,timbre_avg3_flatness,timbre_avg4_attack,timbre_avg5,timbre_avg6,timbre_avg7,timbre_avg8,timbre_avg9,timbre_avg10,timbre_avg11,timbre_avg12
timbre_avg1_loudness,13.950435,-106.186348,103.197243,45.481994,0.983189,46.672694,13.244442,8.790600,3.030430,3.788196,-10.168937,-13.512756
timbre_avg2_brightness,-106.186348,1552.246136,-380.751116,-210.898442,-506.756755,-236.996437,4.000217,-145.309555,-131.621418,-102.074129,17.275475,-40.263819
timbre_avg3_flatness,103.197243,-380.751116,1622.115632,396.968241,16.118980,514.379704,539.299178,-55.400931,-51.778912,81.555749,-171.015033,-342.217142
timbre_avg4_attack,45.481994,-210.898442,396.968241,1008.457850,-10.310300,24.828029,104.040068,455.120136,52.335601,-120.700319,-172.353592,-103.371923
timbre_avg5,0.983189,-506.756755,16.118980,-10.310300,785.880503,-70.416653,184.300372,55.348780,385.595531,190.869584,-36.853908,101.223232
timbre_avg6,46.672694,-236.996437,514.379704,24.828029,-70.416653,641.108707,101.348142,13.933532,-137.019132,188.522922,-186.830168,-164.926703
timbre_avg7,13.244442,4.000217,539.299178,104.040068,184.300372,101.348142,559.296259,9.428276,91.721396,117.138678,-62.847153,-9.311587
timbre_avg8,8.790600,-145.309555,-55.400931,455.120136,55.348780,13.933532,9.428276,416.149417,25.467470,-39.409811,-70.997883,13.795578
timbre_avg9,3.030430,-131.621418,-51.778912,52.335601,385.595531,-137.019132,91.721396,25.467470,500.509609,103.735208,-51.871608,48.860842
timbre_avg10,3.788196,-102.074129,81.555749,-120.700319,190.869584,188.522922,117.138678,-39.409811,103.735208,227.162875,-58.314995,-4.042818
